### Cây quyết định. Thuật toán ID3

Sử dụng các thư viện

In [1]:
import pandas as pd
import math as mt
from collections import Counter
from sklearn.model_selection import train_test_split

Hàm tính toán Entropy

In [2]:
# Function to calculate the entropy of probaility of observations
def entropy(probs):
    return sum([-prob * mt.log(prob, 2) for prob in probs])

Hàm tính toán Entropy với đầu vào là một dãy

Hàm Counter đầu ra là một dict bao gồm keys là phần tử, value là số lần xuất hiện của phần tử đó trong dãy.

In [3]:
# Function to calulate the entropy of the given Data Sets/List with respect to target attributes
def entropy_of_list(a_list):
    cnt = Counter(x for x in a_list)
    num_instances = len(a_list)
    probs = [x/num_instances for x in cnt.values()]
    return entropy(probs)

Hàm tính toán information gain của thuộc tính

Sử dụng các hàm trong pandas: groupby, agg

In [4]:
# Information gain of Attributes
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    # split data by possible vals of attribute
    df_split = df.groupby(split_attribute_name)
    # proportion of Obs in Each data_split
    nobs = len(df.index)
    df_agg_ent = df_split.agg({target_attribute_name: [entropy_of_list, lambda x: len(x) / nobs]})[
        target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    # Calculate Information Gain:
    new_entropy = sum(df_agg_ent['Entropy'] * df_agg_ent['PropObservations'])
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy

Thuật toán ID3

In [5]:
# ID3 Algorithm
def id3_algorithm(df, target_attribute_name, attribute_names, default_class=None):
    cnt = Counter(x for x in df[target_attribute_name])
    if len(cnt) == 1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        # Get Default Value for next recursive call of this function:
        default_class = max(cnt.keys())
        # Compute the information gain of the attribute:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names]
        # index of best attribute
        index_of_max = gainz.index(max(gainz))
        # choose best attribute to split on
        best_attr = attribute_names[index_of_max]
        # create an empty tree, to be populated in a moment
        # Iniiate the tree with best attribute as a node
        tree = {best_attr: {}}
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        # Split dataset
        # On each split, recursively call this algorithm.
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3_algorithm(data_subset, target_attribute_name, remaining_attribute_names, default_class)
            tree[best_attr][attr_val] = subtree
        return tree

Độ chính xác của cây quyết định

In [6]:
# Classification accuracy
def classify(instance, tree, default=None):
    # instance of play tennis with predict
    attribute = next(iter(tree))
    if instance[attribute] in tree[attribute].keys():  # Value of the attributs in  set of Tree keys
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict):  # this is a tree, delve deeper
            return classify(instance, result)
        else:
            return result  # this is a label
    else:
        return default

### Bài toán dự đoán người mắc bệnh tiểu đường

Dữ liệu gồm 768 bản ghi, 9 trường dữ liệu gồm thông tin về bệnh tiểu đường

Các trường dữ liệu bao gồm: ố lần mang thai,huyết áp, độ dày, insulin huyết thanh, chỉ số khối cơ thể, phả hệ, tuổi và phân lớp.

In [7]:
df_data = pd.read_csv("pima-indians-diabetes.csv")
df_data.head(5)

,Number,Plasma glucose,Diastolic blood,Triceps skin,2-Hour serum,Body mass,Diabetes,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Number           768 non-null    int64  
 1   Plasma glucose   768 non-null    int64  
 2   Diastolic blood  768 non-null    int64  
 3   Triceps skin     768 non-null    int64  
 4   2-Hour serum     768 non-null    int64  
 5   Body mass        768 non-null    float64
 6   Diabetes         768 non-null    float64
 7   Age              768 non-null    int64  
 8   Class            768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


ID3 decision tree là nó có thể làm việc với các đặc trưng dạng categorical, thường là rời rạc và không có thứ tự.

Cần tiền xử lý dữ liệu: xóa bỏ 2 thuộc tính có kiểu dữ liệu float là: Body mass và Diabetes.

Lấy tên các đặc trưng

In [9]:
 attribute_names = list(df_data.columns)

In [10]:
 attribute_names

['Number',
 'Plasma glucose',
 'Diastolic blood',
 'Triceps skin',
 '2-Hour serum',
 'Body mass',
 'Diabetes',
 'Age',
 'Class']

In [11]:
attribute_names.remove('Class')
attribute_names.remove('Body mass')
attribute_names.remove('Diabetes')

In [12]:
 print("Predicting Attributes:", attribute_names)

Predicting Attributes: ['Number', 'Plasma glucose', 'Diastolic blood', 'Triceps skin', '2-Hour serum', 'Age']


Chia dữ liệu thành hai tập training và test với tỉ lệ 8:2

In [13]:
training_data, test_data = train_test_split(df_data, test_size=0.2)
print(training_data)

     Number  Plasma glucose  Diastolic blood  Triceps skin  2-Hour serum  \
182       1               0               74            20            23   
349       5               0               80            32             0   
394       4             158               78             0             0   
423       2             115               64            22             0   
207       5             162              104             0             0   
..      ...             ...              ...           ...           ...   
692       2             121               70            32            95   
646       1             167               74            17           144   
709       2              93               64            32           160   
103       1              81               72            18            40   
267       2             128               64            42             0   

     Body mass  Diabetes  Age  Class  
182       27.7     0.299   21      0  
349      

Hình thành cây quyết định

In [14]:
train_tree = id3_algorithm(training_data, 'Class', attribute_names)
print(train_tree)

{'Plasma glucose': {0: {'Number': {1: 0, 5: 1}}, 44: 0, 56: 0, 57: 0, 61: 0, 62: 0, 65: 0, 67: 0, 68: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: {'Number': {1: 0, 3: 1, 6: 0}}, 81: 0, 82: 0, 83: 0, 84: {'Number': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 8: 0, 12: 1}}, 85: {'Number': {1: 0, 2: 0, 4: 0, 5: 1, 6: 0, 11: 0}}, 86: 0, 87: 0, 88: {'Diastolic blood': {30: 1, 58: 0, 62: 0, 66: 0, 74: 0, 78: 0}}, 89: 0, 90: {'Triceps skin': {0: 0, 8: 0, 12: 0, 14: 0, 17: 0, 18: 0, 42: 1, 47: 0}}, 91: 0, 92: {'Number': {1: 0, 2: 0, 4: 0, 6: 0, 12: 1}}, 93: {'Number': {0: 0, 1: 0, 2: 1, 6: 0}}, 94: 0, 95: {'Diastolic blood': {54: 0, 64: {'Number': {0: 0, 4: 1}}, 66: 0, 70: 0, 72: 0, 82: 1, 85: 1}}, 96: 0, 97: {'Number': {0: 0, 1: 0, 4: 0, 5: 1, 7: 1}}, 98: 0, 99: 0, 100: {'Number': {0: 0, 1: 0, 2: 0, 3: 0, 7: 1, 8: {'Diastolic blood': {74: 1, 76: 0}}, 12: 0, 14: 1}}, 101: {'Diastolic blood': {50: 0, 58: 0, 65: 0, 76: 0, 86: 1}}, 102: {'Triceps skin': {0: 1, 17: 0, 23: 0, 36: 1, 39

Dự đoán của cây quyết định

In [15]:
test_data['predicted'] = test_data.apply(classify,axis=1,args=(train_tree, 1))

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
test_data[['Class', 'predicted']]

,Class,predicted
74,0,0.0
355,1,0.0
216,1,0.0
714,0,1.0
649,0,NaN
...,...,...
404,1,1.0
70,1,0.0
187,1,NaN
341,0,NaN


Tính toán độ chính xác của cây quyết định

In [21]:
print(' Accuracy model is : ', 100*sum(test_data['Class'] == test_data['predicted']) / (1.0 * len(test_data.index)), "%")

 Accuracy model is :  46.753246753246756 %


Nhận thấy rằng độ chính xác của mô hình cây quyết định ID3 rất thấp và trong trường dự đoán có các giá trị NAN, bởi vì bộ dữ liệu được lựa chọn tương đối liên tục và các giá trị trong tập test có thể không xuất hiện trong tập train. Nhìn chung cây quyết định dễ mắc phải hiện tượng overfit.